In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time
import cv2 as cv

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/05-grasp'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/05-grasp'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time
import cv2 as cv

#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

#you can also change the shape & size
RealWorld.getFrame("obj0").setColor([1.,0,0])
RealWorld.getFrame("obj0").setShape(ry.ST.sphere, [.03])
# RealWorld.getFrame("obj0").setPosition([0., .2, 1.])
RealWorld.getFrame("obj0").setPosition([0., .2, 2.])

RealWorld.getFrame("obj0").setContact(1)

#remove some objects
for o in range(1,30):
    name = "obj%i" % o
#     print("deleting", name)
    RealWorld.delFrame(name)
    
    
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

# setup your model world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")

obj = C.addFrame("object")
obj.setColor([1.,0,0]) # set the color of one objet to red!
obj.setShape(ry.ST.sphere, [.03]) # assume that we know the object shape

# obj.setQuaternion([1,0,1,0])
# obj.setShape(ry.ST.capsule, [.2,.02])
# obj.setColor([1,0,1])

D = C.view();

In [4]:
# camera parameters
cameraFrame = RealWorld.frame("camera")
camInfo = cameraFrame.info()

f = camInfo['focalLength']; W=camInfo['width']; H=camInfo['height']; #from the scenario file
                
fx, fy = f*H, f*H
px, py = W/2, H/2
R, off = cameraFrame.getRotationMatrix(), cameraFrame.getPosition()
R,off


(array([[ 1.       ,  0.       ,  0.       ],
        [ 0.       ,  0.8660254, -0.5      ],
        [ 0.       ,  0.5      ,  0.8660254]]),
 array([-0.01, -0.2 ,  1.8 ]))

In [5]:
def red_obj_det(image):
    result = image.copy()

    image_hsv = cv.cvtColor(image, cv.COLOR_RGB2HSV)

    # lower boundary RED color range values; Hue (0 - 10)
    lower1 = np.array([0, 100, 20])
    upper1 = np.array([10, 255, 255])

    # upper boundary RED color range values; Hue (160 - 180)
    lower2 = np.array([160,100,20])
    upper2 = np.array([179,255,255])

    lower_mask = cv.inRange(image_hsv, lower1, upper1)
    upper_mask = cv.inRange(image_hsv, lower2, upper2)

    full_mask = lower_mask + upper_mask;

    result = cv.bitwise_and(result, result, mask=full_mask)

    mask_greyscale = np.repeat(full_mask[:,:,np.newaxis],3,axis=2)
    
    contours, hierarchy = cv.findContours(cv.cvtColor(mask_greyscale, cv.COLOR_RGB2GRAY),cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
    return contours, full_mask


def pos_image_to_world(cameraFrame, x, y, Z):
    ## depth is sign-fliped, j: right, i: down
    ## Camera Frame
    pos = np.zeros(3)
    pos[0] = Z * (x - px) / fx;
    pos[1] = -Z * (y - py) / fy;
    pos[2] = -Z
    
    ## Coordinate transformation (from camera to world) 
    pos = (R@pos[:,np.newaxis]).flatten() + off
    
    return pos

def z_from_contour(depth, contour):
    global o_depth, o_contour

    o_depth = depth
    o_contour = contour
    c = contour.reshape(-1,2)
    return depth[c[:,1], c[:,0]].mean()

def estimateBallPosition(depth, contour):
    Z_mean = z_from_contour(depth, contour)
    
    M = cv.moments(contour)
    
    if (M["m00"]):
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    
        pos = pos_image_to_world(cameraFrame, cX, cY, Z_mean)
    else:
        pos = np.zeros(3)
        print("Error: Ball not detected!")
    
    return pos 

In [6]:
def find_moving_ball(prev_contours, prev_full_mask, contours, full_mask):       
    solution = contours[0]

    if prev_contours != None:
        diff_mask = (full_mask - prev_full_mask).clip(0,255)

        for c in contours:
            image_mask = cv.drawContours(np.zeros(full_mask.shape, np.uint8), [c], -1, 255, cv.FILLED)
            combination_mask = cv.bitwise_and(diff_mask, image_mask)
            if (np.count_nonzero(combination_mask)>20):
                solution = c
                break;
    return solution

In [7]:
def solve_komo_path_grasping(waypoints = 5, horizon_seconds =.5):
    komo = C.komo_path(1., waypoints, horizon_seconds, False)

    # encode grasping
    komo.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    1e2*np.array([[1,0,0],[0,1,0]]),
                    [0.,0.,.0]);

    # encode hight tracking
    komo.addObjective([0.,.5], 
                ry.FS.positionDiff, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                1.2e2*np.array([[0,0,1]]),
                [0.,0.,.0]);

    komo.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);

    komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)

    komo.optimize()
#     komo.getReport()
    return komo.getPathFrames() 

def solve_komo_path_lifting(release_pos = [0., 0., 1.5], waypoints = 5, horizon_seconds =.5):
    komo = C.komo_path(1., waypoints, horizon_seconds, False)

    komo.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["R_gripperCenter", "world"], 
                    ry.OT.eq, 
                    1e2,
                    release_pos);


    komo.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);


    komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)

    komo.optimize()
#     komo.getReport()
    return komo.getPathFrames() 

In [8]:
def solve_komo_IK_grasping():
    IK = C.komo_IK(False)

    # encode grasping
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    1e2*np.array([[1,0,0],[0,1,0]]),
                    [0.,0.,.0]);

    # encode hight tracking
    IK.addObjective([0.,.5], 
                ry.FS.positionDiff, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                1.2e2*np.array([[0,0,1]]),
                [0.,0.,.0]);

    IK.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);


    IK.optimize()
#     IK.getReport()
    return IK.getPathFrames()  

def solve_komo_IK_lifting(release_pos = [0., 0., 1.5]):
    IK = C.komo_IK(False)

    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["R_gripperCenter", "world"], 
                    ry.OT.eq, 
                    1e2,
                    release_pos);


    IK.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);


    IK.optimize()
#     IK.getReport()
    return IK.getPathFrames() 

In [9]:
def grasping(KOMO = False):
    points = []
    tau = .01

    gripping = False
    grasped = False

    prev_contours = None
    prev_full_mask = None
    # small cheating, better intitialization
    objectPosition = RealWorld.frame("obj0").getPosition();

    for t in range(3000):
        time.sleep(tau)

        #grab sensor readings from the simulation
        q = S.get_q()
        if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

            contours, full_mask = red_obj_det(rgb)
            if (len(contours)!=0):
                mv_ball_contour = find_moving_ball(prev_contours, prev_full_mask, contours, full_mask)

                objectPosition = estimateBallPosition(depth, mv_ball_contour)
                prev_contours, prev_full_mask = contours, full_mask

        # TOTAL CHEAT: grab the true position from the RealWorld
    #     objectPosition = RealWorld.frame("obj0").getPosition();
        obj.setPosition(objectPosition) # set the model object to percept

        C.setJointState(q); # set your robot model to match the real q

        y, J = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])

    #     qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)

#         komo_next_state = solve_komo_IK_grasping()[0]
        if KOMO:
            if t%10 == 0:
                komo_next_state = solve_komo_path_grasping()[0]
        else:
            komo_next_state = solve_komo_IK_grasping()[0]
            
        C.setFrameState(komo_next_state)

        qdot = ( C.getJointState() - q ) /  (  40*tau )

        if not gripping and np.linalg.norm(y) < .01:
            S.closeGripper("R_gripper", speed=1.)
            gripping = True

        if gripping and S.getGripperWidth("R_gripper") < 1e-2:
            print("FAILED!")
            S.openGripper("R_gripper", speed=1.)
            gripping = False
            return False

        if gripping and S.getGripperIsGrasping("R_gripper"):
            print("GRASPED!")
            grasped = True
            return True

        S.step(qdot, tau, ry.ControlMode.velocity)
    
    

In [10]:
def lift_release(release_pos = [0., 0., 1.5], released = False, KOMO = False):
    
    points = []
    tau = .01
    
    releasing = False

    for t in range(3000):
        time.sleep(tau)

        #grab sensor readings from the simulation
        q = S.get_q()

        C.setJointState(q); # set your robot model to match the real q

        y, J = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "world"])

    #     qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)


        if KOMO:
            if t%10 == 0:
                komo_next_state = solve_komo_path_lifting(release_pos)[0]
        else:
            komo_next_state = solve_komo_IK_lifting(release_pos)[0]
            
        C.setFrameState(komo_next_state)

        qdot = ( C.getJointState() - q ) /  (  40*tau )

        if not released and np.linalg.norm(y-release_pos) < .01:
            S.openGripper("R_gripper", speed=1.)
            releasing = True
        
        if releasing and S.getGripperWidth("R_gripper") > .03:
            released = True
            releasing = False
            return True

        S.step(qdot, tau, ry.ControlMode.velocity)
    
    return False
    

In [11]:
def lift_trow(trow_pos = [0., 0., 1.2], waypoints = 5, horizon_seconds =.6):
    
    points = []
    tau = .01
    komo_step = 0
    released = False
    releasing = False
    komo_done = False

    for t in range(3000):
        time.sleep(tau)

        #grab sensor readings from the simulation
        q = S.get_q()

        C.setJointState(q); # set your robot model to match the real q

        y, J = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "world"])

    #     qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)

        if ((t*tau)%(horizon_seconds/(waypoints+1)) < 1.e-9) and not komo_done:
            if komo_step == 0:
#                 if t != 0:
#                     return
                komo_next_states = solve_komo_path_lifting(2*np.array(trow_pos))
#                 komo = C.komo_path(1., waypoints, horizon_seconds, False)

#                 komo.addObjective([1.], 
#                                 ry.FS.positionRel, 
#                                 ["R_gripperCenter", "world"], 
#                                 ry.OT.eq, 
#                                 1e2,
#                                 );


#                 komo.addObjective([0.,1.], 
#                                 ry.FS.quaternionRel, 
#                                 ["world", "R_gripperCenter"], 
#                                 ry.OT.eq, 
#                                 [1e2],
#                                 [.579,.406,-.406,.579]);


# #                 komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)

#                 komo.optimize()
#             #     komo.getReport()
#                 komo_next_states = komo.getPathFrames() 

                C.setFrameState(komo_next_states[komo_step])
                komo_step+=1
            else:
                print("komo_step {}".format(komo_step))
                C.setFrameState(komo_next_states[komo_step])
                komo_step+=1
                if (komo_step == waypoints):
                    komo_step = 0
                    komo_done = True
            
            print("Speed calculation")
            qdot = ( C.getJointState() - q ) /  (  40*tau )
        elif t*tau > (horizon_seconds+.1):
            qdot = .1*qdot

        if not released and np.abs(y[2]-trow_pos[2]) < .1:
            print(y)
            print(trow_pos)
            S.openGripper("R_gripper", speed=1.)
            releasing = True
        
        if releasing and S.getGripperWidth("R_gripper") > .03:
            released = True
            releasing = False

        S.step(qdot, tau, ry.ControlMode.velocity)
    
    return released
    

In [12]:
# while (True):
#     grasped = grasping()
#     if not grasped:
#         print("Failiure grasping")
#         break;
#     released = lift_release()
#     if not released:
#         print("Failiure releasing")
#         break;

In [13]:
while (True):
    grasped = grasping()
    if not grasped:
        print("Failiure grasping")
        break;
    released = lift_trow()
    if not released:
        print("Failiure releasing")
        break;

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.059391 (kin:0.005162 coll:0 feat:0.031503 newton: 0.008371) setJointStateCount:1001
   sos:0.101143 ineq:0 eq:98.1499
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.056537 (kin:0.003157 coll:0 feat:0.02863 newton: 0.00876) setJointStateCount:1001
   sos:0.097102 ineq:0 eq:97.977
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.056375 (kin:0.004402 coll:0 feat:0.029229 newton: 0.006309) setJointStateCount:1001
   sos:0.0920621 ineq:0 eq:98.1682
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.051735 (kin:0.006166 coll:0 feat:0.028637 ne

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.073179 (kin:0.003284 coll:0 feat:0.039041 newton: 0.004063) setJointStateCount:1001
   sos:0.0195415 ineq:0 eq:98.6651
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.047661 (kin:0.002368 coll:0 feat:0.029284 newton: 0.00304) setJointStateCount:1000
   sos:0.0192285 ineq:0 eq:97.8925
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.048254 (kin:0.002626 coll:0 feat:0.030891 newton: 0.0031) setJointStateCount:1000
   sos:0.018671 ineq:0 eq:97.2486
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.046335 (kin:0.002551 coll:0 feat:0.025083 n

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000754 (kin:3.7e-05 coll:0 feat:0.000302 newton: 6.9e-05) setJointStateCount:8
   sos:0.0129094 ineq:0 eq:0.0157005
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000689 (kin:3e-05 coll:0 feat:0.000283 newton: 6.9e-05) setJointStateCount:8
   sos:0.012235 ineq:0 eq:0.015421
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000586 (kin:2.7e-05 coll:0 feat:0.000225 newton: -6.02768) setJointStateCount:7
   sos:0.0116045 ineq:0 eq:0.0501497
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.001018 (kin:4.3e-05 coll:0 feat:0.000401 newton: -6

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000964 (kin:4.1e-05 coll:0 feat:0.000334 newton: -13.7884) setJointStateCount:4
   sos:0.00345776 ineq:0 eq:0.0726231
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000438 (kin:1.7e-05 coll:0 feat:0.000142 newton: -13.8608) setJointStateCount:4
   sos:0.0032862 ineq:0 eq:0.059031
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000453 (kin:1.6e-05 coll:0 feat:0.000146 newton: -13.9236) setJointStateCount:4
   sos:0.00312755 ineq:0 eq:0.0403124
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000331 (kin:1.3e-05 coll:0 feat:0.000122 new

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.0008 (kin:3.3e-05 coll:0 feat:0.000275 newton: -7.76608) setJointStateCount:5
   sos:0.00519025 ineq:0 eq:0.0261658
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.001032 (kin:4.1e-05 coll:0 feat:0.000408 newton: -7.80908) setJointStateCount:5
   sos:0.0049377 ineq:0 eq:0.0235974
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000804 (kin:3.4e-05 coll:0 feat:0.000296 newton: -7.84744) setJointStateCount:5
   sos:0.00469704 ineq:0 eq:0.0202613
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000553 (kin:2.4e-05 coll:0 feat:0.000192 newt

...

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000461 (kin:1.9e-05 coll:0 feat:0.000143 newton: -10.4345) setJointStateCount:3
   sos:0.000436929 ineq:0 eq:0.0182284
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.00032 (kin:1.1e-05 coll:0 feat:9.6e-05 newton: -10.4622) setJointStateCount:3
   sos:0.000416616 ineq:0 eq:0.0205483
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000317 (kin:1.1e-05 coll:0 feat:9.7e-05 newton: -10.4702) setJointStateCount:3
   sos:0.000392535 ineq:0 eq:0.0210381
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000318 (kin:1.2e-05 coll:0 feat:9.6e-05 ne

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000549 (kin:1.4e-05 coll:0 feat:0.000255 newton: -11.4017) setJointStateCount:3
   sos:9.67424e-05 ineq:0 eq:0.0927071
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000424 (kin:1.6e-05 coll:0 feat:0.000126 newton: -11.4352) setJointStateCount:3
   sos:8.91951e-05 ineq:0 eq:0.0163243
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000833 (kin:2.9e-05 coll:0 feat:0.000261 newton: -11.4826) setJointStateCount:3
   sos:8.18093e-05 ineq:0 eq:0.0162851
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000278 (kin:9e-06 coll:0 feat:8.4e-05 n

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.00036 (kin:1.4e-05 coll:0 feat:0.000114 newton: 2.4e-05) setJointStateCount:3
   sos:2.15346e-05 ineq:0 eq:0.0233144
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000358 (kin:1.3e-05 coll:0 feat:0.000114 newton: 2.4e-05) setJointStateCount:3
   sos:2.02597e-05 ineq:0 eq:0.0358469
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000533 (kin:1.8e-05 coll:0 feat:0.000183 newton: -12.5396) setJointStateCount:3
   sos:2.12578e-05 ineq:0 eq:0.0328608
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000511 (kin:1.5e-05 coll:0 feat:0.000159 n

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000818 (kin:3e-05 coll:0 feat:0.000258 newton: -13.6643) setJointStateCount:3
   sos:2.12899e-05 ineq:0 eq:0.0222871
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000708 (kin:4.3e-05 coll:0 feat:0.000205 newton: -27.3744) setJointStateCount:2
   sos:2.39238e-05 ineq:0 eq:0.102719
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000362 (kin:1.1e-05 coll:0 feat:0.000102 newton: -13.7802) setJointStateCount:2
   sos:1.90282e-05 ineq:0 eq:0.0223267
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000448 (kin:1.7e-05 coll:0 feat:0.000142 n

KeyboardInterrupt: 

In [ ]:
# lift_release([0,0,.7],False)

In [ ]:
lift_trow()